<a href="https://colab.research.google.com/github/darisoy/EE517_Sp21/blob/master/Project/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up the environment

In [1]:
# install packages
%%capture
!pip install spacy==2.2.4
!python -m spacy download en
!venv .env
!source .env/bin/activate
!git clone https://github.com/huggingface/neuralcoref.git
!pip install -r /content/neuralcoref/requirements.txt
!pip install -e /content/neuralcoref/

!pip install allennlp
!pip install allennlp-models

!pip install gender-guesser
!git clone https://github.com/darisoy/EE517_Sp21.git
!pip install pyspotlight

In [1]:
import pandas as pd
from allennlp.predictors.predictor import Predictor
import gender_guesser.detector as gender
from tqdm.notebook import tqdm
import spotlight

# load models
import spacy
nlp = spacy.load('en')

import neuralcoref
# restart runtime if this throws an error
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')
genDec = gender.Detector()

allen_model_url = 'https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz'
allen_predictor = Predictor.from_path(allen_model_url)

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
100%|██████████| 40155833/40155833 [00:00<00:00, 42509399.97B/s]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Plugin allennlp_models could not be loaded: No module named 'nltk.translate.meteor_score'
downloading: 100%|##########| 1345947288/1345947288 [00:40<00:00, 33159098.18B/s]


Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
# list of person mentions that are not names
fem_p = ['male', 'she', 'her', 'hers', 'herself']
male_p = ['female', 'he', 'him', 'his', 'himself']
personal_p = ['i', 'me', 'we', 'our', 'us', 'myself', 'ourself', 'ourselves']
other_p = ['they', 'them', 'their', 'you', 'your', 'themself', 'themselves']
people = ['people', 'adult','adults', 'person','people','child','children']

person_f_singular = ['girl','woman','mrs','ms','mother','mom','aunt','niece','sister','wife','daughter','grandmother','grandma','grandmom','granddaughter','bride','girlfriend','gal','madam','lady']
person_m_singular = ['boy','man','mr','father','dad','uncle','nephew','brother','husband','son','grandfather','grandpa','granddad','grandson','groom','boyfriend','guy','gentleman','bachelor']
people_f_plural = ['females', 'girls','women','mothers','moms','aunts','nieces','sisters','wives','daughters','grandmothers','grandmas','granddaughters','brides','girlfriends','gals','ladies']
people_m_plural = ['males', 'boys','men','fathers','dads','uncles','nephews','brothers','husbands','sons','grandfathers','grandpas','grandsons','grooms','boyfriends','guys','gentlemen','bachelors']
people_f = person_f_singular + people_f_plural
people_m = person_m_singular + people_m_plural

un_named_mentions = fem_p + male_p + personal_p + other_p + people + people_f + people_m
f_un_named = fem_p + people_f
m_un_named = male_p + people_m
gendered_un_named = f_un_named + m_un_named
neutral_un_named = personal_p + other_p + people

In [ ]:
# NO NEED TO RUN
# get the raw data
train = pd.read_csv('/content/train.csv', delimiter= ",", low_memory=False, index_col=0)
test = pd.read_csv('/content/test.csv', delimiter= ",", low_memory=False, index_col=0)
data = pd.concat([train, test], axis = 0)
data = data.drop(['bool', 'book', 'text', 'level'], axis=1)
data = data.rename(columns={'text_org': 'text'})
data.fillna('[]',inplace = True)
data.reset_index(drop=True, inplace=True)
data[data.grade=='12'].head()

# Helper Functions

In [91]:
# run text through models

def get_coref(text, allen=False):
    if allen:
        text_idx = [[word.text, word.idx, word.idx + len(word)] for word in nlp(text)]
        prediction = allen_predictor.predict_tokenized([word.text for word in nlp(text)])
        all_clusters = []
        for cluster in prediction['clusters']:
            cluster_mentions = []
            for (token_idx_start, token_idx_end) in cluster:
                if len(text_idx[token_idx_start: token_idx_end + 1]) > 1:
                    start = text_idx[token_idx_start][1]
                    end = text_idx[token_idx_end][2]
                    text_mention = text[start : end]
                    text_start_end = [text_mention, start, end]
                else:
                    text_start_end = text_idx[token_idx_start: token_idx_end + 1][0]
                cluster_mentions.append([text_start_end[1], text_start_end[2]-1])
            all_clusters.append(cluster_mentions)
        return all_clusters
    else:
        doc = nlp(text)
        coref = [[[ent.start_char, ent.end_char-1] for ent in cluster.mentions] for cluster in doc._.coref_clusters]
        return coref

def get_ner(text):
    return [[ent.start_char, ent.end_char-1] for ent in nlp(text).ents if ent.label_ == 'PERSON']

def get_nel(text, threshold=0.8):
    results = []
    try:
        results = spotlight.annotate('https://api.dbpedia-spotlight.org/en/annotate', text)
    except:
        result = []
    return [[p['offset'], p['offset']+len(p['surfaceForm'])-1] for p in results if 'Person' in p['types'] and float(p['similarityScore']) >= threshold]

def get_gender(person_mention):
    if person_mention.lower() in gendered_un_named:
        if person_mention.lower() in f_un_named:
            return 'F'
        elif person_mention.lower() in m_un_named:
            return 'M'
    elif len(person_mention) > 0:
        first_name = person_mention.split()[0]
        gen_name = genDec.get_gender(first_name.capitalize())
        if 'female' in gen_name:
            return 'F'
        elif 'male' in gen_name:
            return 'M'
    return '-'

def get_pronouns(text):
    doc = nlp(text)
    result = []
    for word in doc:
        if word.text.lower() in gendered_un_named:
            result.append([word.idx, word.idx+len(word.text)-1])
    return result

In [21]:
# converting spans to text

def span_to_string(span, dic):
    [a, b] = span
    return "".join(dic[a:b+1])

def array_to_text(array, dic):
    return [span_to_string(e, dic) for e in array]

def array2d_to_text(array2D, dic):
    return [array_to_text(arr, dic) for arr in array2D]

In [95]:
# get hypo & histo from model results

def get_cluster(person, clusters):
    for i, cluster in enumerate(clusters):
        if person in cluster:
            return clusters[i]
    return [person]

def split_hist_hypo(clusters, person_mentions, famous_people, gender):
    historical = []
    hypothetical = []
    hist_genders = []
    hypo_genders = []
    for i, person in enumerate(person_mentions):
        # skip if not gendered
        if gender[i] == '-':
            continue
        # skip if already in one of the lists
        in_historical = any(person in sublist for sublist in historical)
        in_hypothetical = any(person in sublist for sublist in hypothetical)
        if in_historical or in_hypothetical:
            continue
        # add the cluster to correct list
        person_set = get_cluster(person, clusters)
        if any(p in person_set and p in person_mentions for p in famous_people):
            historical.append(person_set)
            hist_genders.append(gender[i])
        else:
            hypothetical.append(person_set)
            hypo_genders.append(gender[i])
    return historical, hist_genders, hypothetical, hypo_genders

In [96]:
# pipline for a single given text
def get_hist_hypo_references(text, allen=False, debug=False):
    clusters = get_coref(text, allen=allen)
    person_mentions = get_ner(text) + get_pronouns(text)
    gender = [get_gender(span_to_string(p, text)) for p in person_mentions]
    famous_people = get_nel(text)
    hist, hist_g, hypo, hypo_g = split_hist_hypo(clusters, person_mentions, famous_people, gender)
    hist_str = array2d_to_text(hist, text)
    hypo_str = array2d_to_text(hypo, text)
    if debug and len(person_mentions) > 0:
        print('***NEW SENTENCE***\t\t %s' % text)
        print()
        print('Coreference Model Output\t %s' % array2d_to_text(clusters, text))
        print('NER + Pronouns Output\t\t %s' % array_to_text(person_mentions, text))
        print('Gender Guesser Output\t\t %s' % gender)
        print('NEL Output\t\t\t %s' % famous_people)
        print()
        print('Historical references\t\t %s' % hist_str)
        print('Historical genders\t\t %s' % hist_g)
        print('Hypothetical references\t\t %s' % hypo_str)
        print('Hypothetical genders\t\t %s' % hypo_g)
        print()
        print()
    return hist, hist_g, hypo, hypo_g, dic, famous_people

In [98]:
sample = 'Isaac Newton invented the wheel. He didn\'t go to kindergarden but he was familiar with circles. When told this story, Jessica didn\'t believe it. She thought Newton was a lie.'
hist, hist_g, hypo, hypo_g, dic, database = get_hist_hypo_references(sample, debug=True)

***NEW SENTENCE***		 Isaac Newton invented the wheel. He didn't go to kindergarden but he was familiar with circles. When told this story, Jessica didn't believe it. She thought Newton was a lie.

Coreference Model Output	 [['Isaac Newton', 'He', 'he', 'Newton'], ['this story', 'it'], ['Jessica', 'She']]
NER + Pronouns Output		 ['Isaac Newton', 'Jessica', 'Newton', 'He', 'he', 'She']
Gender Guesser Output		 ['M', 'F', 'M', 'M', 'M', 'F']
NEL Output			 [[0, 11], [157, 162]]

Historical references		 [['Isaac Newton', 'He', 'he', 'Newton']]
Historical genders		 ['M']
Hypothetical references		 [['Jessica', 'She']]
Hypothetical genders		 ['F']




# Run pipeline

In [ ]:
# NO NEED TO RUN
# Save only gendered sentences as csv file
data_genders = [False for _ in range(len(data))]
for i, row in tqdm(data.iterrows()):
    dic = row.text.split()
    person_mentions = get_ner(row.text, nlp) + get_pronouns(dic)
    genders = [get_gender(span_to_string(p, dic)) for p in person_mentions]
    male_mentions = [_ for g in genders if g == 'M']
    female_mentions = [_ for g in genders if g == 'F']
    data_genders[i] = len(male_mentions)>0 or len(female_mentions)>0
data['gendered'] = data_genders
gendered = data[data['gendered']]
gendered = gendered.drop(['gendered'], axis=1)
gendered.to_csv('/content/gendered.csv', index=False)

In [9]:
gendered_data = pd.read_csv('/content/gendered.csv', delimiter= ",", low_memory=False)

In [14]:
sci_hist_male = 0
sci_hist_female = 0
sci_hypo_male = 0
sci_hypo_female = 0
for i, row in tqdm(gendered_data[gendered_data.science == 1].iterrows()):
    _, hist_g, _, hypo_g, _, _ = get_hist_hypo_references(row.text)
    sci_hist_male += len([g for g in hist_g if g == 'M'])
    sci_hist_female += len([g for g in hist_g if g == 'F'])
    sci_hypo_male += len([g for g in hypo_g if g == 'M'])
    sci_hypo_female += len([g for g in hypo_g if g == 'F'])

In [15]:
print("Scientific Test")
print("\tReal male count: %d" % sci_hist_male)
print("\tReal female count: %d" % sci_hist_female)
print("\tHypothetical male count: %d" % sci_hypo_male)
print("\tHypothetical female count: %d" % sci_hypo_female)

Scientific Test
	Real male count: 381
	Real female count: 50
	Hypothetical male count: 1514
	Hypothetical female count: 1209


In [16]:
nosci_hist_male = 0
nosci_hist_female = 0
nosci_hypo_male = 0
nosci_hypo_female = 0
for i, row in tqdm(gendered_data[gendered_data.science == 0].iterrows()):
    _, hist_g, _, hypo_g, _, _ = get_hist_hypo_references(row.text)
    nosci_hist_male += len([g for g in hist_g if g == 'M'])
    nosci_hist_female += len([g for g in hist_g if g == 'F'])
    nosci_hypo_male += len([g for g in hypo_g if g == 'M'])
    nosci_hypo_female += len([g for g in hypo_g if g == 'F'])

In [17]:
print("Non-Scientific Test")
print("\tReal male count: %d" % nosci_hist_male)
print("\tReal female count: %d" % nosci_hist_female)
print("\tHypothetical male count: %d" % nosci_hypo_male)
print("\tHypothetical female count: %d" % nosci_hypo_female)

Non-Scientific Test
	Real male count: 832
	Real female count: 131
	Hypothetical male count: 1190
	Hypothetical female count: 625


# Evaluate Annotations

In [ ]:
%cd /content/EE517_Sp21/
!git pull

## Gender guesser evaluations

In [ ]:
pipe_male = 0
pipe_female = 0
pipe_total = 0
ann_male = 0
ann_female = 0
ann_total = 0
pipe_ann_total = 0
pipe_ann_male = 0
pipe_ann_female = 0
for i in range(100):
    text_file = "/content/EE517_Sp21/Project/F_M_N_mentions/F_M_N_%d.txt" % (i+1)
    ann_file = "/content/EE517_Sp21/Project/F_M_N_mentions/F_M_N_%d.ann" % (i+1)
    
    text = open(text_file, 'r').readline() 
    dic = [t.text for t in nlp(text)]

    person_mentions = get_ner(text, nlp) + get_pronouns(dic, gendered=False)
    genders = [get_gender(span_to_string(p, dic)) for p in person_mentions]
    pipe_male_mentions = [_ for g in genders if g == 'M']
    pipe_female_mentions = [_ for g in genders if g == 'F']
    pipe_total += len(person_mentions)
    pipe_male += len(pipe_male_mentions)
    pipe_female += len(pipe_female_mentions)

    pipe_mentions = array_to_text(person_mentions, dic)

    for line in open(ann_file, 'r'):
        info = line.split('\t')
        gender = info[1].split()[0]
        word = info[2][:-1]

        if gender == "Male":
            ann_male += 1
        if gender == "Female":
            ann_female += 1
        ann_total += 1

        if word in pipe_mentions:
            pipe_ann_total += 1
            index = pipe_mentions.index(word)
            if genders[index] == gender[0]:
                if gender == "Male":
                    pipe_ann_male += 1
                if gender == "Female":
                    pipe_ann_female += 1

print("Pipeline results:")
print("\tTotal person mentions: %i" % pipe_total)
print("\tMale mentions: %i" % pipe_male)
print("\tFemale mentions: %i" % pipe_female)
print()
print("Annotations results:")
print("\tTotal person mentions: %i" % ann_total)
print("\tMale mentions: %i" % ann_male)
print("\tFemale mentions: %i" % ann_female)
print()
print("In pipeline and annotations results:")
print("\tTotal person mentions: %i" % pipe_ann_total)
print("\tMale mentions: %i" % pipe_ann_male)
print("\tFemale mentions: %i" % pipe_ann_female)

Pipeline results:<br>
> Total person mentions: 320<br>
> Male mentions: 54<br>
> Female mentions: 20<br>

Annotations results: <br>
> Total person mentions: 292<br>
> Male mentions: 64<br>
> Female mentions: 17<br>

In pipeline and annotations results:<br>
> Total person mentions: 274<br>
> Male mentions: 48<br>
> Female mentions: 16<br>

In [ ]:
print("Overall:")
overall_precision = (pipe_ann_total / pipe_total) * 100
print("\tPrecision: %.2f%%" % overall_precision)
overall_recall = (pipe_ann_total / ann_total) * 100
print("\tRecall: %.2f%%" % overall_recall)
overall_f = (2 * overall_precision * overall_recall) / (overall_precision + overall_recall)
print("\tF-score: %.2f%%" % overall_f)
print()
print("Male:")
male_precision = (pipe_ann_male / pipe_male) * 100
print("\tPrecision: %.2f%%" % male_precision)
male_recall = (pipe_ann_male / ann_male) * 100
print("\tRecall: %.2f%%" % male_recall)
male_f = (2 * male_precision * male_recall) / (male_precision + male_recall)
print("\tF-score: %.2f%%" % male_f)
print()
print("Female:")
female_precision = (pipe_ann_female / pipe_female) * 100
print("\tPrecision: %.2f%%" % female_precision)
female_recall = (pipe_ann_female / ann_female) * 100
print("\tRecall: %.2f%%" % female_recall)
female_f = (2 * female_precision * female_recall) / (female_precision + female_recall)
print("\tF-score: %.2f%%" % female_f)

Overall:<br>
> Precision: 85.62%<br>
> Recall: 93.84%<br>
> F-score: 89.54%<br>

Male: <br>
> Precision: 88.89%<br>
> Recall: 75.00%<br>
> F-score: 81.36%<br>

Female: <br>
> Precision: 80.00%<br>
> Recall: 94.12%<br>
> F-score: 86.49%<br>


## Coref and NEL evaluations (combined)

In [ ]:
# DO NOT RUN THIS ANYMORE ON ACCIDENT
# generate and save pre-annotation .ann files
for i in tqdm(range(300)):
    # text_file = "/content/EE517_Sp21/Project/Coref_NEL_mentions/Coref_NEL_%d.txt" % (i+1)
    # ann_file = "/content/EE517_Sp21/Project/Coref_NEL_mentions/Coref_NEL_%d.ann" % (i+1)
    text = open(text_file, 'r').readline()
    hist, _, hypo, _, dic, database = get_hist_hypo_references(text, nlp, nel)
    generate_ann_file(text, hist, hypo, dic, database, ann_file)

In [12]:
# dictinories for evaluations

def get_pipe_dic(text):
    hist, _, hypo, _, dic, database = get_hist_hypo_references(text)
    in_database = []
    for cluster in hist:
        in_database += [w for w in cluster if span_to_string(w, dic) in database]
    return array2d_to_text(hist+hypo, dic), array_to_text(in_database, dic)

def get_ann_dic(file):
    ann_dic = {}
    in_database = []
    for line in file:
        tabs = line.split('\t')
        key = tabs[1].split()[0]
        if len(tabs) < 3:
            continue
        value = tabs[2]
        if value[-1] == '\n':
            value = value[:-1]
        if key == "InDatabase":
            in_database.append(value)
            continue
        if key in ann_dic.keys():
            current = ann_dic[key]
            current.append(value)
            ann_dic.update({ key: current})
        else:
            ann_dic.update({ key: [value]})
    return list(ann_dic.values()), in_database

In [93]:
def get_all_links(clusters):
    all_links_by_mention = []
    for cluster in clusters:
        for mention in cluster:
            mention_based = []
            for other_mention in cluster:
                mention_based.append((mention,other_mention))
            all_links_by_mention.append(mention_based)
    return all_links_by_mention

In [104]:
def get_precision_recall_macro(answer_mention_links, output_mention_links, get_score):
    '''
    get_socre : "precision" or "recall"
    '''
    if get_score == "recall":
        links1 = answer_mention_links
        links2 = output_mention_links
    elif get_score == "precision":
        links1 = output_mention_links
        links2 = answer_mention_links
    else:
        print("Indicate precision or recall!")
        return 
    
    score = 0
    for mention_links in links1:
        if mention_links in links2:
            compare_links =  links2[mention]
            correct_links = [link for link in mention_links if link in compare_links]
            score += len(correct_links)/len(mention_links)
        else:
            score += 0
    return score/len(links1)

In [111]:
nel_total = 0
nel_ann = 0
nel_pipe = 0
answer_mention_links = []
output_mention_links = []
for i in tqdm(range(300)):
    text_file = "/content/EE517_Sp21/Project/Coref_NEL_mentions/Coref_NEL_%d.txt" % (i+1)
    ann_file = "/content/EE517_Sp21/Project/Coref_NEL_mentions/Coref_NEL_%d.ann" % (i+1)
    text = open(text_file, 'r').readline()
    
    pipe_dic, pipe_database = get_pipe_dic(text, nlp)
    nel_pipe += len(pipe_database)
    ann_dic, ann_database = get_ann_dic(open(ann_file, 'r'))
    nel_ann += len(ann_database)
    nel_total += len([p for p in pipe_database if p in ann_database])

    answer_mention_links.append(get_all_links(ann_dic))
    output_mention_links.append(get_all_links(pipe_dic))

    print(i+1)
    print(pipe_dic)
    # print(pipe_database)
    print(ann_dic)
    # print(ann_database)
    print()

1
[['MS']]
[]

2
[['daughter']]
[['daughter']]

3
[['Mark Pagel'], ['Stephen Jay Gould'], ['ed'], ['ed'], ['Colleen Whitney'], ['Kate Barton'], ['David Smith'], ['Dave Smith'], ['David Ulansey'], ['David Ulansey'], ['Nova Online'], ['Richard Cowen'], ['Roger Perkins'], ['Roy Caldwell'], ['David Lindberg']]
[['Mark Pagel'], ['Stephen Jay Gould'], ['Colleen Whitney'], ['Kate Barton'], ['', 'David Smith'], ['Lexi Krock'], ['Richard Cowen'], ['Roger Perkins'], ['Roy Caldwell'], ['Dave Smith'], ['David Ulansey'], ['David Lindberg']]

4
[['Taft', 'he', 'he', 'his', 'He', 'his', 'He', 'his', 'his', 'him', 'Taft', 'President Taft', 'He', 'he', 'his'], ['Roosevelt', 'Roosevelt', 'Roosevelt', 'Roosevelt', 'Roosevelt', 'Roosevelt'], ['man']]
[['Taft', 'man', 'he', 'he', 'He', 'his', 'He', 'his', 'him', 'Taft', 'President Taft', 'He', 'he', 'his'], ['Roosevelt', 'Roosevelt', 'Roosevelt', 'Roosevelt', 'Roosevelt', 'his', 'Roosevelt']]

5
[['Daniel Fahrenheit', 'his'], ['Anders Celsius']]
[['Daniel 

In [105]:
print("In pipeline and annotations NELs: %i" % nel_total)
print("Annotation NELs: %i" % nel_ann)
print("Pipeline NELs: %i" % nel_pipe)
print()
precision = (nel_total / nel_pipe) * 100
print("Precision: %.2f%%" % precision)
recall = (nel_total / nel_ann) * 100
print("Recall: %.2f%%" % recall)
f = (2 * precision * recall) / (precision + recall)
print("F-score: %.2f%%" % f)

recall_macro = get_precision_recall_macro(answer_mention_links, output_mention_links, "recall")
precision_macro = get_precision_recall_macro(answer_mention_links, output_mention_links, "precision")
print("B-cubed, mention-based approach")
print("Precision : ", precision_macro)
print("Recall : ", recall_macro)
print("F-score : ",  get_f_score(precision_macro, recall_macro))

In pipeline and annotations NELs: 14
Annotation NELs: 21
Pipeline NELs: 16

Precision: 87.50%
Recall: 66.67%
F-score: 75.68%


NameError: ignored